In [ ]:
# create dummy 1000 circles from scikit

from sklearn.datasets import make_circles

# make 1000 circles
n_samples = 1000

# create circles - make a larger circle containing a smaller one in 2D
X,y = make_circles(
    n_samples,
    noise=0.03,
    random_state=42 # for reproducibility
)

In [ ]:
# the features
X[:10]

In [ ]:
# the labels
y[:10]

`visualize the data`

In [ ]:
import pandas as pd 
circles = pd.DataFrame({"X0":X[:, 0], "X1":X[:, 1], "label":y})
# get the first axis for X1 and the second axis for X2
circles.head(5)

In [ ]:
import matplotlib.pyplot as plt 
plt.scatter(X[:,0], X[:,1],c=y,cmap=plt.cm.RdYlBu)

`check the shapes of our features and labels`

In [ ]:
X.shape, y.shape # y are scalers

In [ ]:
len(X), len(y)

In [ ]:
# what is our first example like
X[5], y[5]
# given the two inputs, can we get the output - classify as a zero or a one

`Build a Neural Net to Classify either into blue or red circle`

In [ ]:
import tensorflow as tf 
import numpy as np

In [ ]:
# convert our numpy array into tensors
X = tf.constant(X)
y = tf.constant(y)

In [ ]:
# X,y # they are now tensors
X.dtype, y.dtype

`create our datasets by indexing`

In [ ]:
X_train = X[:800]
y_train = y[:800]

X_test = X[800:]
y_test = y[800:]

plt.scatter(X_train[:,0], X_train[:,1],c=y_train,cmap=plt.cm.RdYlBu)
plt.scatter(X_test[:,0], X_test[:,1], c=y_test,cmap=plt.cm.BuGn)

In [ ]:
# # using sckit to split our data - pops dtype error
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
# 1. set random seed
tf.random.set_seed(42)

# 2. create a model using the Sequential API
model = tf.keras.Sequential(
    [
        # tf.keras.Input(shape=(224,224,3)),
        tf.keras.layers.Dense(100, activation='relu'), # rectifier linear unit
        tf.keras.layers.Dense(1, activation='sigmoid') # output layer
    ]
) 

# 3. Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.035), 
    metrics = 'accuracy' # accuracy - out of 100 examples, how many did our model get right?
)

# 3b Instantiate an EarlyStopping callback
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)

# 4. fit the model
model_def = model.fit(X_train, y_train, epochs=150, verbose=0, callbacks=[callback])

In [ ]:
# see for how long the thing ran before being callback was initiated
len(model_def.history['accuracy'])

In [ ]:
# plot the training before stopping occured
pd.DataFrame(model_def.history).plot(figsize=(7,4))
plt.xlabel('epochs')
plt.ylabel('param to monitor')

In [ ]:
model.summary()
model.evaluate(X_test, y_test) # do not evaluate on the model that you have trained on

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
plt.scatter(X_train[:,0], X_train[:,1],c=y_train,cmap=plt.cm.RdYlBu)
plt.scatter(X_test[:,0], X_test[:,1],c=y_pred,cmap=plt.cm.RdYlBu)

In [ ]:
# save this model in a h5 format
model.save('circles.h5')

`to better understand the performance of the model`

In [ ]:
def plot_decision_boundary(model, X, y):
    """
    Plot the decision boundaries created by a model predicting on X
    """

    # define the axis boundaries of the plot and create a meshgrid
    x_min, x_max = X[:,0].min()-0.1, X[:,0].max()+0.1
    y_min, y_max = X[:,1].min()-0.1, X[:,1].max()+0.1
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 100),
        np.linspace(y_min, y_max, 100)
    )
    
    # create X value - this is where we are going to make predictions on
    x_in = np.c_[xx.ravel(), yy.ravel()] #np.c_ - stacks 2D arrays together

    # make predictions
    y_pred_m = model.predict(x_in)

    # check for multiclass
    if len(y_pred_m[0]) > 1:
        print("performing multiclass classification")
        # therefore, we reshape our predictions and ready them for plotting
        y_pred_m = np.argmax(y_pred_m, axis=1).reshape(xx.shape)
    else:
        print("perfoming binary classification")
        y_pred_m = np.round(y_pred_m).reshape(xx.shape)

    # plot the decision boundary
    plt.contourf(xx, yy, y_pred_m, cmap=plt.cm.RdYlBu, alpha=1)
    plt.scatter(X[:,0], X[:,1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

In [ ]:
# # check the prediction our model is making
# plot_decision_boundary(
#     model=model,
#     X=X_test,
#     y=y_test
# )

`create a learning rate callback`

In [ ]:
# set random seed
tf.random.set_seed(42)

# Create a model 
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(7, activation='relu'),
        tf.keras.layers.Dense(9, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)

# Compile the model
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer= tf.keras.optimizers.Adam(), # we are going to set up a lr callback
    metrics = 'accuracy'
)

# Create a LR callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20))
# Create an EarlyStopping callback
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=5)

# Pass in lr_scheduler into the fit
# Fit the model
model_sc = model.fit(X_train, y_train, epochs=1000, verbose=0, callbacks=[lr_scheduler,early_stop])

In [ ]:
# check for how many times it ran
len(model_sc.history['accuracy'])

In [ ]:
# check out the entire history
pd.DataFrame(model_sc.history).plot(figsize=(7,5), xlabel='epochs')